<a href="https://colab.research.google.com/github/dxvsh/LearningPytorch/blob/main/Week7/DLP_GA7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DLP GA7


Instructions
- pip install datasets transformers evaluate jiwer
- import torch, import datasets, import evaluate, import numpy as np, from dataclasses import data class, field. from typing import Any, Dict, List, Optional, Union
- load train and test splits of mozilla-foundation/common voice 11 0” assamese dataset
- for test, take only first 10 examples and proceed (otherwise you will hit Out of Memory Error)
- from transformers import WhisperFeatureExtractor
- Load feature extractor from the pretrained whisper tiny of openai
- from transformers import WhisperTokenizer
- load tokenizer of openai whisper tiny for assamese for asr task
- from transformers import WhisperProcessor
- load processor of openai whisper tiny for assamese for asr task
- from datasets import Audio
- convert audio to sampling rate of 16k
- write a function which takes batches of input data and gives batches with features    extracted and corresponding labels from the tokenizer.
- from transformers import WhisperForConditionalGeneration
- model = WhisperForConditionalGeneration.from pretrained(”openai/whisper-tiny”)
- set language to assamese, task to asr and forced decoder ids to none
- Write a data collator
- import evaluate and set metric = evaluate.load(”wer”)
- write a function def compute metrics(pred) which takes predictions and returns wer
- from transformers import Seq2SeqTrainingArguments
- set the arguments for Seq2SeqTrainingArguments() as mentioned in the link
-https://huggingface.co/blog/fine-tune-whisper” but change warmup steps to 50, max steps to 100, per device train batch size=8, per device eval batch size=1, save steps=100, eval steps=100
- from transformers import Seq2SeqTrainer
- Set the arguments based on the info mentioned in the above link
- start training


In [ ]:
!pip install datasets transformers evaluate jiwer > /dev/null

Load the train & test splits of the Assamese config of the [Common Voice Dataset](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0):

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice_as = DatasetDict()

# load the assamese train dataset
common_voice_as['train'] = load_dataset("mozilla-foundation/common_voice_11_0", "as", split="train")

# load the test dataset, take only the first 10 samples from the test dataset
common_voice_as['test'] = load_dataset("mozilla-foundation/common_voice_11_0", "as", split="test[:10]")

In [ ]:
common_voice_as

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 824
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10
    })
})

The dataset contains lots of metadata which is not really required for our use case, so we can remove those columns:

In [ ]:
common_voice_as = common_voice_as.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

In [ ]:
common_voice_as

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 824
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10
    })
})

Now, we're left with only the required features, the audio and its text transcript.

**Q1.** How many examples are present in the train split of mozilla-foundation/common voice 11 0 ”assamese” langauge dataset ?

In [ ]:
len(common_voice_as['train'])

824

There are **824** samples in the "assamese" config of the train split of the mozilla common voice dataset

**Q2.** How many unique characters are there in the train split text ?

In [ ]:
combined_text = " ".join(common_voice_as['train']['sentence']) # combine all the text in the train set into a single string
unique_chars = set(combined_text)
len(unique_chars)

74

In [ ]:
print(unique_chars)

{'“', 'য', 'ব', 'ু', 'ছ', 'ৈ', 'দ', 'আ', '‘', '”', 'ল', 'ঠ', 'ী', '.', 'ঙ', 'ৃ', 'ূ', 'ঔ', 'ো', 'ভ', 'ৎ', 'ধ', 'উ', 'এ', '়', 'ঢ', 'ঁ', 'ং', 'ঃ', 'ট', 'স', 'া', 'ঞ', 'ষ', 'ম', 'য়', 'জ', 'গ', 'ত', 'ৌ', '’', 'প', 'ই', '৷', '্', 'খ', 'ক', 'ঢ়', 'ৰ', ',', 'ড়', '-', "'", 'ঈ', 'অ', 'ন', 'হ', 'ফ', 'ৱ', 'ড', 'ি', 'থ', 'ও', 'ঘ', 'শ', '"', 'ণ', 'র', '!', 'চ', ' ', '।', 'ে', '?'}


There are **74** unique characters in the train split of the dataset.

**Q3.** What is the sampling rate of the original mozilla-foundation/common voice 11 0 ”assamese” language audio in Hz?

Let's take a look at a sample from the dataset:

In [ ]:
common_voice_as['train'][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/fdcfd174c1db561f74a5aab292ff32458ceffd67c10de1ac5f5b77eae211090c/as_train_0/common_voice_as_22074894.mp3',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -1.51620850e-06, -1.28747206e-06, -6.44893703e-07]),
  'sampling_rate': 48000},
 'sentence': 'দেখিলে যে অসমীয়া মানুহৰ জ্ঞান-উন্নতি পিনে অলপাে মনকাণ নাই'}

We can see that the original sampling rate is **48000** and the format of the audio files is **mp3**.

In [ ]:
common_voice_as['train'].features

{'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

**Q4.** What is the format of the mozilla-foundation/common voice 11 0 ”assamese” language audio ?

- [x] A. mp3
- [ ] B. M4A
- [ ] C. wav
- [ ] D. FLAC




---



### Load whisper feature extractor

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

### Load whisper tokenizer

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="Assamese", task="transcribe")

**Q5.** What will be the window length in msec if n fft is 400 in ”WhisperFeatureExtractor” ?

In [ ]:
feature_extractor

WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [ ]:
window_length (ms) = (400 / 16000) * 1000 = 25 ms

**Q6.** What is the first token number after tokenising the 56th example?

In [ ]:
input_str = common_voice_as["train"][56]["sentence"]
labels = tokenizer(input_str).input_ids
print(labels)
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


[50258, 50350, 50359, 50363, 29045, 103, 29045, 122, 29045, 101, 156, 11061, 29045, 97, 220, 29045, 237, 156, 100, 108, 29045, 123, 220, 29045, 101, 29045, 123, 29045, 99, 29045, 123, 220, 156, 2250, 29045, 123, 29045, 110, 156, 100, 108, 220, 29045, 103, 29045, 122, 156, 100, 108, 29045, 97, 220, 29045, 98, 29045, 243, 29045, 122, 220, 29045, 237, 29045, 253, 29045, 122, 220, 29045, 114, 29045, 123, 29045, 110, 29045, 97, 220, 29045, 98, 156, 100, 229, 29045, 243, 156, 100, 229, 29045, 248, 29045, 123, 220, 29045, 106, 29045, 122, 156, 100, 108, 29045, 123, 220, 29045, 110, 29045, 122, 29045, 117, 156, 100, 229, 29045, 243, 156, 8667, 220, 29045, 244, 29045, 122, 29045, 229, 220, 29045, 98, 29045, 107, 156, 15773, 8703, 97, 50257]
Input:                 পানীত এৰি নিদি বিলৰ পাৰত থকা এটা শিলত থেকেচি মাৰি লাহেকৈ খাই থয়।
Decoded w/ special:    <|startoftranscript|><|as|><|transcribe|><|notimestamps|>পানীত এৰি নিদি বিলৰ পাৰত থকা এটা শিলত থেকেচি মাৰি লাহেকৈ খাই থয়।<|endoftext|>
Decoded w/

In [ ]:
labels[0]

50258

The first token number after tokenizing the 56th sample is 50258.

**Q7.** What is the token corresponding to the token number 51833 in whisper?
- [ ] A. < |89.38| >
- [x] B. < |29.38| >
- [ ] C. < |9.38| >
- [ ] D. < |19.38| >


In [ ]:
tokenizer.convert_ids_to_tokens(51833)

'<|29.38|>'

**Q8.** Is token number 51833 a special token?
- [ ] A. Yes
- [x] B. No

In [ ]:
# this list contains all the special tokens, so we can use it to make the check
tokenizer.all_special_ids

[50257,
 50258,
 50259,
 50260,
 50261,
 50262,
 50263,
 50264,
 50265,
 50266,
 50267,
 50268,
 50269,
 50270,
 50271,
 50272,
 50273,
 50274,
 50275,
 50276,
 50277,
 50278,
 50279,
 50280,
 50281,
 50282,
 50283,
 50284,
 50285,
 50286,
 50287,
 50288,
 50289,
 50290,
 50291,
 50292,
 50293,
 50294,
 50295,
 50296,
 50297,
 50298,
 50299,
 50300,
 50301,
 50302,
 50303,
 50304,
 50305,
 50306,
 50307,
 50308,
 50309,
 50310,
 50311,
 50312,
 50313,
 50314,
 50315,
 50316,
 50317,
 50318,
 50319,
 50320,
 50321,
 50322,
 50323,
 50324,
 50325,
 50326,
 50327,
 50328,
 50329,
 50330,
 50331,
 50332,
 50333,
 50334,
 50335,
 50336,
 50337,
 50338,
 50339,
 50340,
 50341,
 50342,
 50343,
 50344,
 50345,
 50346,
 50347,
 50348,
 50349,
 50350,
 50351,
 50352,
 50353,
 50354,
 50355,
 50356,
 50357,
 50358,
 50359,
 50360,
 50361,
 50362,
 50363]

In [ ]:
51833 in tokenizer.all_special_ids

False

**Q9.** What is the token corresponding to the token number 50350 in whisper?
- [ ] A. < |asp| >
- [ ] B. < |s| >
- [x] C. < |as| >
- [ ] D. < |pa| >


In [ ]:
tokenizer.convert_ids_to_tokens(50350)

'<|as|>'

**Q10.** Is token number 50350 a special token?
- [x] A. Yes
- [ ] B. No


In [ ]:
50350 in tokenizer.all_special_ids

True